In [ ]:
# Import libraries
import keras
from keras.datasets import mnist

# Import datasets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [ ]:
# Code source: https://bitbucket.org/tarlanahad/myneatcodes/src/master/RBFNN/Python/

# Import libraries
import numpy as np
import time

# The Euclidean Distance between two points
def euc_dist(x1, x2):
    return np.sqrt(np.sum((x1-x2)**2))

def kmeans(X, k, max_iters):
    centroids = X[np.random.choice(range(len(X)), k, replace=False)]
    # centroids = [np.random.uniform(size=len(X[0])) for i in range(k)]

    converged = False
    current_iter = 0

    while (not converged) and (current_iter < max_iters):

        cluster_list = [[] for i in range(len(centroids))]

        for x in X:  # Go through each data point
            distances_list = []
            for c in centroids:
                distances_list.append(euc_dist(c, x))
            cluster_list[int(np.argmin(distances_list))].append(x)

        cluster_list = list((filter(None, cluster_list)))

        prev_centroids = centroids.copy()

        centroids = []

        for j in range(len(cluster_list)):
            centroids.append(np.mean(cluster_list[j], axis=0))

        pattern = np.abs(np.sum(prev_centroids) - np.sum(centroids))

        print('K-MEANS: ', int(pattern))

        converged = (pattern == 0)

        current_iter += 1

    return np.array(centroids), [np.std(x) for x in cluster_list]

class RBF:

    def __init__(self, X, y, tX, ty, num_of_classes, k, std_from_clusters=True):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.k = k
        self.std_from_clusters = std_from_clusters

    def convert_to_one_hot(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr

    def get_rbf(self, x, c, s):
        distance = euc_dist(x, c)
        return 1 / np.exp(-distance / s ** 2)

    def get_rbf_as_list(self, X, centroids, std_list):
        RBF_list = []
        for x in X:
            RBF_list.append([self.get_rbf(x, c, s) for (c, s) in zip(centroids, std_list)])
        return np.array(RBF_list)

    def fit(self):

        self.centroids, self.std_list = kmeans(self.X, self.k, 1000)

        if not self.std_from_clusters:
            dMax = np.max([euc_dist(c1, c2) for c1 in self.centroids for c2 in self.centroids])
            self.std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

        RBF_X = self.get_rbf_as_list(self.X, self.centroids, self.std_list)

        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.convert_to_one_hot(self.y, self.number_of_classes)

        RBF_list_tst = self.get_rbf_as_list(self.tX, self.centroids, self.std_list)

        self.pred_ty = RBF_list_tst @ self.w

        self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

        diff = self.pred_ty - self.ty

        print('Accuracy: ', len(np.where(diff == 0)[0]) / len(diff))

In [ ]:
RBF_CLASSIFIER = RBF(x_train, y_train, x_test, y_test, num_of_classes=10, k=10, std_from_clusters=False)

In [ ]:
RBF_CLASSIFIER.fit()

K-MEANS:  43922
K-MEANS:  14
K-MEANS:  9799
K-MEANS:  7893
K-MEANS:  5339
K-MEANS:  3842
K-MEANS:  2566
K-MEANS:  1688
K-MEANS:  1082
K-MEANS:  728
K-MEANS:  425
K-MEANS:  348
K-MEANS:  244
K-MEANS:  187
K-MEANS:  205
K-MEANS:  122
K-MEANS:  101
K-MEANS:  99
K-MEANS:  75
K-MEANS:  77
K-MEANS:  39
K-MEANS:  53
K-MEANS:  31
K-MEANS:  8
K-MEANS:  13
K-MEANS:  6
K-MEANS:  13
K-MEANS:  13
K-MEANS:  16
K-MEANS:  1
K-MEANS:  10
K-MEANS:  22
K-MEANS:  8
K-MEANS:  0
K-MEANS:  3
K-MEANS:  4
K-MEANS:  1
K-MEANS:  5
K-MEANS:  2
K-MEANS:  2
K-MEANS:  0
K-MEANS:  1
K-MEANS:  0
K-MEANS:  0
K-MEANS:  1
K-MEANS:  0
Accuracy:  0.6661


In [ ]:
RBF_CLASSIFIER = RBF(x_train, y_train, x_test, y_test, num_of_classes=10, k=100, std_from_clusters=False)

start = time.time()
RBF_CLASSIFIER.fit()
end = time.time()
print('Fit duration: %.6f ms' % (1000.0*(end-start)))

K-MEANS:  89116
K-MEANS:  136248
K-MEANS:  49259
K-MEANS:  17546
K-MEANS:  6670
K-MEANS:  5094
K-MEANS:  4811
K-MEANS:  3534
K-MEANS:  2183
K-MEANS:  776
K-MEANS:  139
K-MEANS:  1055
K-MEANS:  1479
K-MEANS:  1069
K-MEANS:  779
K-MEANS:  563
K-MEANS:  848
K-MEANS:  948
K-MEANS:  315
K-MEANS:  252
K-MEANS:  109
K-MEANS:  615
K-MEANS:  353
K-MEANS:  88
K-MEANS:  134
K-MEANS:  637
K-MEANS:  534
K-MEANS:  87
K-MEANS:  646
K-MEANS:  228
K-MEANS:  943
K-MEANS:  748
K-MEANS:  356
K-MEANS:  419
K-MEANS:  553
K-MEANS:  463
K-MEANS:  606
K-MEANS:  395
K-MEANS:  17
K-MEANS:  388
K-MEANS:  51
K-MEANS:  132
K-MEANS:  124
K-MEANS:  16
K-MEANS:  42
K-MEANS:  51
K-MEANS:  65
K-MEANS:  125
K-MEANS:  59
K-MEANS:  27
K-MEANS:  49
K-MEANS:  163
K-MEANS:  245
K-MEANS:  52
K-MEANS:  37
K-MEANS:  15
K-MEANS:  427
K-MEANS:  493
K-MEANS:  285
K-MEANS:  656
K-MEANS:  236
K-MEANS:  47
K-MEANS:  144
K-MEANS:  108
K-MEANS:  98
K-MEANS:  26
K-MEANS:  139
K-MEANS:  91
K-MEANS:  127
K-MEANS:  13
K-MEANS:  70
K-MEANS: 

In [ ]:
RBF_CLASSIFIER = RBF(x_train, y_train, x_test, y_test, num_of_classes=10, k=200, std_from_clusters=False)

start = time.time()
RBF_CLASSIFIER.fit()
end = time.time()
print('Fit duration: %.6f ms' % (1000.0*(end-start)))

K-MEANS:  99827
K-MEANS:  390107
K-MEANS:  85360
K-MEANS:  42067
K-MEANS:  22278
K-MEANS:  11864
K-MEANS:  8321
K-MEANS:  3723
K-MEANS:  2438
K-MEANS:  494
K-MEANS:  832
K-MEANS:  549
K-MEANS:  464
K-MEANS:  354
K-MEANS:  555
K-MEANS:  199
K-MEANS:  297
K-MEANS:  553
K-MEANS:  770
K-MEANS:  2036
K-MEANS:  1058
K-MEANS:  1492
K-MEANS:  1394
K-MEANS:  130
K-MEANS:  467
K-MEANS:  669
K-MEANS:  495
K-MEANS:  293
K-MEANS:  69
K-MEANS:  303
K-MEANS:  65
K-MEANS:  912
K-MEANS:  718
K-MEANS:  74
K-MEANS:  91
K-MEANS:  52
K-MEANS:  12
K-MEANS:  358
K-MEANS:  113
K-MEANS:  300
K-MEANS:  458
K-MEANS:  203
K-MEANS:  153
K-MEANS:  706
K-MEANS:  159
K-MEANS:  455
K-MEANS:  86
K-MEANS:  21
K-MEANS:  160
K-MEANS:  168
K-MEANS:  137
K-MEANS:  109
K-MEANS:  8
K-MEANS:  284
K-MEANS:  255
K-MEANS:  99
K-MEANS:  149
K-MEANS:  12
K-MEANS:  120
K-MEANS:  6
K-MEANS:  22
K-MEANS:  11
K-MEANS:  32
K-MEANS:  37
K-MEANS:  150
K-MEANS:  69
K-MEANS:  135
K-MEANS:  38
K-MEANS:  118
K-MEANS:  109
K-MEANS:  174
K-MEAN

In [ ]:
# 2nd RBF MODEL 

# Import libraries
import keras
from keras.datasets import mnist

# Import datasets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28*28)/255.0
x_test = x_test.reshape(-1, 28*28)/255.0

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes=10)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [ ]:
# PCA Tranformation
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(x_train)
# Apply transform to both the training set and the test set.
train_img = scaler.transform(x_train)
test_img = scaler.transform(x_test)

pca = PCA(0.95)
pca.fit(train_img)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

print("original shape:   ", x_train.shape)
print("transformed shape:", x_train_pca.shape)
print("original shape:   ", x_test.shape)
print("transformed shape:", x_test_pca.shape)

original shape:    (60000, 784)
transformed shape: (60000, 331)
original shape:    (10000, 784)
transformed shape: (10000, 331)


In [ ]:
# 2nd RBF MODEL 
# Code source: https://github.com/PetraVidnerova/rbf_keras

from keras import backend as K
from tensorflow.python.keras.layers import Layer, InputSpec
from keras.initializers import RandomUniform, Initializer, Constant
import numpy as np

from keras.initializers import Initializer
from sklearn.cluster import KMeans


class InitCentersKMeans(Initializer):
    """ Initializer for initialization of centers of RBF network
        by clustering the given data set.
    # Arguments
        X: matrix, dataset
    """

    def __init__(self, X, max_iter=100):
        self.X = X
        self.max_iter = max_iter

    def __call__(self, shape, dtype=None):
        assert shape[1] == self.X.shape[1]

        n_centers = shape[0]
        km = KMeans(n_clusters=n_centers, max_iter=self.max_iter, verbose=0)
        km.fit(self.X)
        return km.cluster_centers_


class InitCentersRandom(Initializer):
    """ Initializer for initialization of centers of RBF network
        as random samples from the given data set.
    # Arguments
        X: matrix, dataset to choose the centers from (random rows
          are taken as centers)
    """

    def __init__(self, X):
        self.X = X

    def __call__(self, shape, dtype=None):
      assert shape[1] == self.X.shape[1]
      idx = np.random.randint(self.X.shape[0], size=shape[0])

      # type checking to access elements of data correctly
      if type(self.X) == np.ndarray:
          return self.X[idx, :]
      elif type(self.X) == pd.core.frame.DataFrame:
          return self.X.iloc[idx, :]


class RBFLayer(Layer):
    """ Layer of Gaussian RBF units.
    # Example
    ```python
        model = Sequential()
        model.add(RBFLayer(10,
                           initializer=InitCentersRandom(X),
                           betas=1.0,
                           input_shape=(1,)))
        model.add(Dense(1))
    ```
    # Arguments
        output_dim: number of hidden units (i.e. number of outputs of the
                    layer)
        initializer: instance of initiliazer to initialize centers
        betas: float, initial value for betas
    """

    def __init__(self, output_dim, initializer=None, betas=1.0, **kwargs):
        self.output_dim = output_dim
        self.init_betas = betas
        if not initializer:
            self.initializer = RandomUniform(0.0, 1.0)
        else:
            self.initializer = initializer
        super(RBFLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        self.centers = self.add_weight(name='centers',
                                       shape=(self.output_dim, input_shape[1]),
                                       initializer=self.initializer,
                                       trainable=True)
        self.betas = self.add_weight(name='betas',
                                     shape=(self.output_dim,),
                                     initializer=Constant(
                                         value=self.init_betas),
                                     # initializer='ones',
                                     trainable=True)

        super(RBFLayer, self).build(input_shape)

    def call(self, x):

        C = K.expand_dims(self.centers)
        H = K.transpose(C-K.transpose(x))
        return K.exp(-self.betas * K.sum(H**2, axis=1))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        # have to define get_config to be able to use model_from_json
        config = {
            'output_dim': self.output_dim
        }
        base_config = super(RBFLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
# Use Grid Search to find the best parameters
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from tensorflow import keras
import pandas as pd
 
# Function to create model, required for KerasClassifier
def create_model(input_shape, num_classes, rbf_neurons, initializer, betas):
  # create model
  model = keras.Sequential(
      [
        keras.layers.Input(shape=input_shape),
        RBFLayer(output_dim=rbf_neurons, initializer=initializer, betas=betas, name="rbf_layer"),
        keras.layers.Dense(num_classes, activation="sigmoid", name="output_layer"),
      ]
  )
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  return model

# Function to find the best parameters for the rbf
def best_param_selection(X, y):
    num_of_classes = 10
    Rbf_neurons = [10, 20, 50]
    Betas = [0.1, 1.0, 2.0]
    Initializer = [InitCentersKMeans(X), InitCentersRandom(X)]
    Batch_size = [16, 32, 64]
    Epochs = [10, 20, 50]

    param_grid = {'betas': Betas, 'initializer': Initializer, 'batch_size': Batch_size, 'epochs': Epochs, 'rbf_neurons': Rbf_neurons}
    
    Estimator = KerasClassifier(build_fn=create_model,input_shape=X[0].shape, num_classes=num_of_classes,  rbf_neurons=Rbf_neurons, verbose=2)
    
    grid_search = GridSearchCV(estimator=Estimator, param_grid=param_grid, verbose=2, cv=3)  
    grid_search.fit(X, y)
    print(grid_search.best_params_)
    return grid_search

grid_search = best_param_selection(x_train_pca, y_train)
print(grid_search.best_params_)
pd.DataFrame(grid_search.cv_results_)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


2022-01-09 05:27:16.761980: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-09 05:27:16.762278: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-09 05:27:16.762304: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kubuntu-2004): /proc/driver/nvidia/version does not exist
2022-01-09 05:27:16.763297: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-09 05:28:03.659002: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 52960000 excee

Epoch 1/10
2500/2500 - 3s - loss: 1.9828 - accuracy: 0.4037
Epoch 2/10
2500/2500 - 3s - loss: 1.6133 - accuracy: 0.5906
Epoch 3/10
2500/2500 - 3s - loss: 1.3875 - accuracy: 0.6521
Epoch 4/10
2500/2500 - 3s - loss: 1.2425 - accuracy: 0.6998
Epoch 5/10
2500/2500 - 3s - loss: 1.1343 - accuracy: 0.7242
Epoch 6/10
2500/2500 - 3s - loss: 1.0470 - accuracy: 0.7477
Epoch 7/10
2500/2500 - 3s - loss: 0.9737 - accuracy: 0.7652
Epoch 8/10
2500/2500 - 3s - loss: 0.9122 - accuracy: 0.7786
Epoch 9/10
2500/2500 - 3s - loss: 0.8619 - accuracy: 0.7902
Epoch 10/10
2500/2500 - 3s - loss: 0.8226 - accuracy: 0.7969
1250/1250 - 1s - loss: 0.7922 - accuracy: 0.8117
[CV] END batch_size=16, betas=0.1, epochs=10, initializer=<__main__.InitCentersKMeans object at 0x7fe2dea7c340>, rbf_neurons=10; total time= 1.5min


2022-01-09 05:29:31.701489: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 52960000 exceeds 10% of free system memory.


Epoch 1/10
2500/2500 - 3s - loss: 2.0502 - accuracy: 0.3701
Epoch 2/10
2500/2500 - 2s - loss: 1.6849 - accuracy: 0.5359
Epoch 3/10
2500/2500 - 3s - loss: 1.4503 - accuracy: 0.6086
Epoch 4/10
2500/2500 - 2s - loss: 1.2755 - accuracy: 0.6700
Epoch 5/10
2500/2500 - 3s - loss: 1.1518 - accuracy: 0.7135
Epoch 6/10
2500/2500 - 3s - loss: 1.0567 - accuracy: 0.7432
Epoch 7/10
2500/2500 - 3s - loss: 0.9824 - accuracy: 0.7610
Epoch 8/10
2500/2500 - 2s - loss: 0.9186 - accuracy: 0.7768
Epoch 9/10
2500/2500 - 3s - loss: 0.8675 - accuracy: 0.7870
Epoch 10/10
2500/2500 - 3s - loss: 0.8214 - accuracy: 0.7983
1250/1250 - 1s - loss: 0.8433 - accuracy: 0.7872
[CV] END batch_size=16, betas=0.1, epochs=10, initializer=<__main__.InitCentersKMeans object at 0x7fe2dea7c340>, rbf_neurons=10; total time= 1.2min


...

2022-01-09 05:32:05.330159: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 52960000 exceeds 10% of free system memory.


Epoch 1/10
2500/2500 - 4s - loss: 1.8143 - accuracy: 0.5155
Epoch 2/10
2500/2500 - 3s - loss: 1.2926 - accuracy: 0.7022
Epoch 3/10
2500/2500 - 3s - loss: 1.0620 - accuracy: 0.7527
Epoch 4/10
2500/2500 - 3s - loss: 0.9141 - accuracy: 0.7880
Epoch 5/10
2500/2500 - 3s - loss: 0.8172 - accuracy: 0.8066
Epoch 6/10
2500/2500 - 4s - loss: 0.7448 - accuracy: 0.8213
Epoch 7/10
2500/2500 - 3s - loss: 0.6916 - accuracy: 0.8319
Epoch 8/10
2500/2500 - 3s - loss: 0.6484 - accuracy: 0.8396
Epoch 9/10
2500/2500 - 3s - loss: 0.6129 - accuracy: 0.8471
Epoch 10/10
2500/2500 - 3s - loss: 0.5840 - accuracy: 0.8532
1250/1250 - 1s - loss: 0.5792 - accuracy: 0.8523
[CV] END batch_size=16, betas=0.1, epochs=10, initializer=<__main__.InitCentersKMeans object at 0x7fe2dea7c340>, rbf_neurons=20; total time= 1.5min


2022-01-09 05:33:45.199146: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 52960000 exceeds 10% of free system memory.


Epoch 1/10
2500/2500 - 3s - loss: 1.7491 - accuracy: 0.5710
Epoch 2/10
2500/2500 - 3s - loss: 1.2719 - accuracy: 0.7213
Epoch 3/10
2500/2500 - 3s - loss: 1.0541 - accuracy: 0.7650
Epoch 4/10
2500/2500 - 4s - loss: 0.9169 - accuracy: 0.7915
Epoch 5/10
2500/2500 - 3s - loss: 0.8219 - accuracy: 0.8073
Epoch 6/10
2500/2500 - 3s - loss: 0.7523 - accuracy: 0.8209
Epoch 7/10
2500/2500 - 3s - loss: 0.6960 - accuracy: 0.8315
Epoch 8/10
2500/2500 - 3s - loss: 0.6546 - accuracy: 0.8386
Epoch 9/10
2500/2500 - 3s - loss: 0.6211 - accuracy: 0.8464
Epoch 10/10
2500/2500 - 3s - loss: 0.5898 - accuracy: 0.8522
1250/1250 - 1s - loss: 0.6179 - accuracy: 0.8227
[CV] END batch_size=16, betas=0.1, epochs=10, initializer=<__main__.InitCentersKMeans object at 0x7fe2dea7c340>, rbf_neurons=20; total time= 1.8min
Epoch 1/10
2500/2500 - 3s - loss: 1.7846 - accuracy: 0.5474
Epoch 2/10
2500/2500 - 4s - loss: 1.2859 - accuracy: 0.7055
Epoch 3/10
2500/2500 - 4s - loss: 1.0532 - accuracy: 0.7509
Epoch 4/10
2500/2500 -

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_betas,param_epochs,param_initializer,param_rbf_neurons,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,76.109561,8.977027,1.011723,0.098175,16,0.1,10,<__main__.InitCentersKMeans object at 0x7fe2de...,10,"{'batch_size': 16, 'betas': 0.1, 'epochs': 10,...",0.81170,0.78715,0.79375,0.797533,0.010373,65
1,99.093506,6.001022,1.109142,0.007680,16,0.1,10,<__main__.InitCentersKMeans object at 0x7fe2de...,20,"{'batch_size': 16, 'betas': 0.1, 'epochs': 10,...",0.85230,0.82270,0.86955,0.848183,0.019347,46
2,174.864988,0.946223,1.698473,0.021613,16,0.1,10,<__main__.InitCentersKMeans object at 0x7fe2de...,50,"{'batch_size': 16, 'betas': 0.1, 'epochs': 10,...",0.88785,0.87220,0.87550,0.878517,0.006736,30
3,36.913890,6.307726,1.129648,0.193590,16,0.1,10,<__main__.InitCentersRandom object at 0x7fe2de...,10,"{'batch_size': 16, 'betas': 0.1, 'epochs': 10,...",0.72860,0.76275,0.79945,0.763600,0.028931,75
4,32.814805,0.098739,1.102095,0.005855,16,0.1,10,<__main__.InitCentersRandom object at 0x7fe2de...,20,"{'batch_size': 16, 'betas': 0.1, 'epochs': 10,...",0.86080,0.84270,0.82285,0.842117,0.015499,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,176.783057,1.987072,0.711152,0.009256,64,2.0,50,<__main__.InitCentersKMeans object at 0x7fe2de...,20,"{'batch_size': 64, 'betas': 2.0, 'epochs': 50,...",0.58365,0.43510,0.44915,0.489300,0.066962,109
158,413.920719,10.188660,1.578193,0.017700,64,2.0,50,<__main__.InitCentersKMeans object at 0x7fe2de...,50,"{'batch_size': 64, 'betas': 2.0, 'epochs': 50,...",0.64295,0.54430,0.73445,0.640567,0.077647,90
159,73.003485,6.615348,0.581463,0.095337,64,2.0,50,<__main__.InitCentersRandom object at 0x7fe2de...,10,"{'batch_size': 64, 'betas': 2.0, 'epochs': 50,...",0.28855,0.30610,0.10905,0.234567,0.089042,150
160,115.795489,0.630362,0.730446,0.007893,64,2.0,50,<__main__.InitCentersRandom object at 0x7fe2de...,20,"{'batch_size': 64, 'betas': 2.0, 'epochs': 50,...",0.43985,0.28825,0.37340,0.367167,0.062047,126


In [ ]:
print(grid_search.best_params_)
pd.DataFrame(grid_search.cv_results_)

{'batch_size': 16, 'betas': 0.1, 'epochs': 50, 'initializer': <__main__.InitCentersRandom object at 0x7fe2dea7cf70>, 'rbf_neurons': 50}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_betas,param_epochs,param_initializer,param_rbf_neurons,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,76.109561,8.977027,1.011723,0.098175,16,0.1,10,<__main__.InitCentersKMeans object at 0x7fe2de...,10,"{'batch_size': 16, 'betas': 0.1, 'epochs': 10,...",0.81170,0.78715,0.79375,0.797533,0.010373,65
1,99.093506,6.001022,1.109142,0.007680,16,0.1,10,<__main__.InitCentersKMeans object at 0x7fe2de...,20,"{'batch_size': 16, 'betas': 0.1, 'epochs': 10,...",0.85230,0.82270,0.86955,0.848183,0.019347,46
2,174.864988,0.946223,1.698473,0.021613,16,0.1,10,<__main__.InitCentersKMeans object at 0x7fe2de...,50,"{'batch_size': 16, 'betas': 0.1, 'epochs': 10,...",0.88785,0.87220,0.87550,0.878517,0.006736,30
3,36.913890,6.307726,1.129648,0.193590,16,0.1,10,<__main__.InitCentersRandom object at 0x7fe2de...,10,"{'batch_size': 16, 'betas': 0.1, 'epochs': 10,...",0.72860,0.76275,0.79945,0.763600,0.028931,75
4,32.814805,0.098739,1.102095,0.005855,16,0.1,10,<__main__.InitCentersRandom object at 0x7fe2de...,20,"{'batch_size': 16, 'betas': 0.1, 'epochs': 10,...",0.86080,0.84270,0.82285,0.842117,0.015499,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,176.783057,1.987072,0.711152,0.009256,64,2.0,50,<__main__.InitCentersKMeans object at 0x7fe2de...,20,"{'batch_size': 64, 'betas': 2.0, 'epochs': 50,...",0.58365,0.43510,0.44915,0.489300,0.066962,109
158,413.920719,10.188660,1.578193,0.017700,64,2.0,50,<__main__.InitCentersKMeans object at 0x7fe2de...,50,"{'batch_size': 64, 'betas': 2.0, 'epochs': 50,...",0.64295,0.54430,0.73445,0.640567,0.077647,90
159,73.003485,6.615348,0.581463,0.095337,64,2.0,50,<__main__.InitCentersRandom object at 0x7fe2de...,10,"{'batch_size': 64, 'betas': 2.0, 'epochs': 50,...",0.28855,0.30610,0.10905,0.234567,0.089042,150
160,115.795489,0.630362,0.730446,0.007893,64,2.0,50,<__main__.InitCentersRandom object at 0x7fe2de...,20,"{'batch_size': 64, 'betas': 2.0, 'epochs': 50,...",0.43985,0.28825,0.37340,0.367167,0.062047,126


In [ ]:
import pandas as pd

results_df = pd.DataFrame(grid_search.cv_results_)
results_df = results_df.sort_values(by=["rank_test_score"])
results_df = results_df.set_index(
    results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("batch_size | beta | epochs | initializer | rbf_neurons")
results_df[["rank_test_score", "mean_test_score", "std_test_score", "mean_fit_time"]].head(20)


,rank_test_score,mean_test_score,std_test_score,mean_fit_time
batch_size | beta | epochs | initializer | rbf_neurons,,,,
16_0.1_50_<__main__.InitCentersRandom object at 0x7fe2dea7cf70>_50,1,0.925683,0.007443,298.528618
16_0.1_50_<__main__.InitCentersKMeans object at 0x7fe2dea7c340>_50,2,0.920083,0.005475,415.794475
32_0.1_50_<__main__.InitCentersKMeans object at 0x7fe2dea7c340>_50,3,0.916583,0.003675,402.880154
64_0.1_50_<__main__.InitCentersKMeans object at 0x7fe2dea7c340>_50,4,0.912567,0.001578,443.215978
32_0.1_50_<__main__.InitCentersRandom object at 0x7fe2dea7cf70>_50,5,0.911883,0.009122,332.513563
64_0.1_50_<__main__.InitCentersRandom object at 0x7fe2dea7cf70>_50,6,0.905500,0.003836,291.775592
16_0.1_50_<__main__.InitCentersKMeans object at 0x7fe2dea7c340>_20,7,0.904433,0.002871,227.788239
16_0.1_20_<__main__.InitCentersKMeans object at 0x7fe2dea7c340>_50,8,0.901233,0.004339,232.782227
16_0.1_20_<__main__.InitCentersRandom object at 0x7fe2dea7cf70>_50,9,0.900917,0.005225,118.740383


In [ ]:
results_df[["rank_test_score", "mean_test_score", "std_test_score", "mean_fit_time"]].tail(10)

,rank_test_score,mean_test_score,std_test_score,mean_fit_time
batch_size | beta | epochs | initializer | rbf_neurons,,,,
16_2.0_10_<__main__.InitCentersKMeans object at 0x7fe2dea7c340>_10,153,0.215967,0.072516,78.644410
32_2.0_10_<__main__.InitCentersRandom object at 0x7fe2dea7cf70>_50,154,0.211833,0.133015,57.188916
32_2.0_10_<__main__.InitCentersRandom object at 0x7fe2dea7cf70>_10,155,0.198017,0.059830,20.105238
64_2.0_20_<__main__.InitCentersRandom object at 0x7fe2dea7cf70>_10,156,0.164283,0.074641,27.778155
64_2.0_10_<__main__.InitCentersKMeans object at 0x7fe2dea7c340>_10,157,0.160050,0.068657,61.781089
64_2.0_10_<__main__.InitCentersRandom object at 0x7fe2dea7cf70>_10,158,0.159700,0.068126,14.189163
32_2.0_10_<__main__.InitCentersKMeans object at 0x7fe2dea7c340>_10,159,0.152583,0.058138,63.254290
16_2.0_10_<__main__.InitCentersRandom object at 0x7fe2dea7cf70>_10,160,0.112400,0.002440,30.513230
64_2.0_20_<__main__.InitCentersKMeans object at 0x7fe2dea7c340>_10,161,0.112367,0.002416,72.272140


In [ ]:
results_df.to_csv('rbf_output.csv')